In [1]:
!pip install transformers
!pip install datasets
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


2023-03-10 16:38:49.973957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 16:38:50.491778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-10 16:38:50.491821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-10 16:38:50.491826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
dataset = pd.read_csv('TatoebaSentencePairsPT-EN-2023-02-15.tsv', sep="\t", header=None, names=["ID-PT", "PT", "ID-EN", "EN"])
dataset = Dataset.from_dict(dataset)
print(dataset)

Dataset({
    features: ['ID-PT', 'PT', 'ID-EN', 'EN'],
    num_rows: 273714
})


In [3]:
labels = ["A1", "A2", "B1", "B2", "C1"]
checkpoint = 'neuralmind/bert-base-portuguese-cased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [4]:
checkpoint = torch.load("COPLE/model_4_acc86%")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
def class_string_to_int(s):
    if s == 'A1':
        return 0
    if s == 'A2':
        return 1
    if s == 'B1':
        return 2
    if s == 'B2':
        return 3
    if s == 'C1':
        return 4
    if s == 'C2':
        return 5

In [6]:
def tokenize_function(example):
    return tokenizer(example["PT"], truncation=True, max_length=512)


tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/274 [00:00<?, ?ba/s]

In [7]:
print(tokenized_dataset)

Dataset({
    features: ['ID-PT', 'PT', 'ID-EN', 'EN', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 273714
})


In [8]:
tokenized_dataset = tokenized_dataset.remove_columns(["ID-PT", "PT", "ID-EN", "EN"])
tokenized_dataset.set_format("torch")
tokenized_dataset.column_names

['input_ids', 'token_type_ids', 'attention_mask']

In [9]:
dataloader = DataLoader(
    tokenized_dataset, batch_size=1, collate_fn=data_collator
)

In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [11]:
model.eval()
total_predictions = []
progress_bar = tqdm(range(len(dataloader)))
for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    total_predictions.append(predictions)
    progress_bar.update(1)

print(total_predictions)

  0%|          | 0/273714 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
preds = [x.item() for x in total_predictions]

In [27]:
df = pd.DataFrame({"pred":preds})
df.head()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 3, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 3, 1, 0, 0, 1, 3, 3, 0, 4, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 0, 0, 3, 1, 1, 3, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 1, 1, 3, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 3, 1, 3, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 3, 1, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 

In [30]:
df.to_csv("tatoeba_preds.csv")

In [32]:
dataset = pd.read_csv('TatoebaSentencePairsPT-EN-2023-02-15.tsv', sep="\t", header=None, names=["ID-PT", "PT", "ID-EN", "EN"])
dataset["Pred"] = df

In [34]:
def class_int_to_string(s):
    if s == 0:
        return 'A1'
    if s == 1:
        return 'A2'
    if s == 2:
        return 'B1'
    if s == 3:
        return 'B2'
    if s == 4:
        return 'C1'
    if s == 5:
        return 'C2'

In [35]:
cefr = [class_int_to_string(x) for x in preds]

In [37]:
dataset["Pred_CEFR"] = cefr

In [40]:
dataset.to_csv("tatoeba_with_preds.csv")